In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/news-odb/news.csv


In [26]:
from tqdm.auto import tqdm, trange
import time
import re

import matplotlib.pyplot as plt
import seaborn as sns

Мы решили торговать на бирже, причем так, чтобы решение о покупке акций принимала нейросеть на основе последних новостей о той или иной компании. Для этого нужно научиться классифицировать все новости. Дана база новостей из разных источников news.csv. Необходимо написать классифицирующую модель новостей по источникам.

Напиши любую модель для предсказания источника новостей и предскажите источники для данных из файла news.csv, для которых отсутствует значение Source.

Пороговые значения для данной задачи классификации считать равными 0.33, 0.66.

In [27]:
data = pd.read_csv('/kaggle/input/news-odb/news.csv', index_col=False)
data = data.drop('Unnamed: 0', axis=1)
data.head()

,news,source
0,Официальный аккаунт PlayStation опубликовал т...,1.0
1,Китайская компания Mobvoi опубликовала на офи...,1.0
2,Практически во всех странах мира введены огра...,0.0
3,"Депутат Госдумы, единоросс Антон Горелкин вне...",2.0
4,Совет директоров «Почты России» проголосовал ...,2.0


In [ ]:
data.info()

In [ ]:
data['source'].value_counts()

In [ ]:
data.isnull().sum()

# Предобработка

In [28]:
!pip install pymystem3

In [29]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

In [30]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
data['preproccessed'] = list(map(preproccessing, data['news']))

In [31]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(data['news'])]

  0%|          | 0/19462 [00:00<?, ?it/s]

In [ ]:
len(prep_text)
prep_text[0]

In [32]:
data['text_prep'] = prep_text

In [33]:
data.head(1)

,news,source,preproccessed,text_prep
0,Официальный аккаунт PlayStation опубликовал т...,1.0,Официальный аккаунт PlayStation опубликовал т...,официальный аккаунт playstation опубликовал т...


In [36]:
data.dtypes

news              object
source           float64
preproccessed     object
text_prep         object
dtype: object

# Стемминг

In [37]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian")

In [38]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [39]:
from nltk.tokenize import word_tokenize
text = data['text_prep'][0]
#word_tokenize(text)

In [40]:
stemmed_texts_list = []
for text in tqdm(data['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

data['text_stem'] = stemmed_texts_list

  0%|          | 0/19462 [00:00<?, ?it/s]

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [43]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(data['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

data['text_sw'] = sw_texts_list

  0%|          | 0/19462 [00:00<?, ?it/s]

In [ ]:
data['text_sw'][0]

# Лемматизация

In [44]:
lemm_texts_list = []
for text in tqdm(data['text_sw']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
data['text_lemm'] = lemm_texts_list

  0%|          | 0/19462 [00:00<?, ?it/s]

In [45]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [46]:
data.to_csv('lemm.csv')

In [53]:
data = pd.read_csv('lemm.csv')

# Модель

In [54]:
train = data[data['source'].notnull()]
test = data[data['source'].isnull()]
test = test.drop(columns = ['Unnamed: 0','source','text_lemm', 'text_stem', 'text_prep', 'preproccessed', 'news'])
train = train.drop(columns = ['Unnamed: 0','text_lemm', 'text_stem', 'text_prep', 'preproccessed', 'news'])

In [55]:
train.head(2), test.head(2)

(   source                                            text_sw
 0     1.0  официальный аккаунт playstation опубликовал тр...
 1     1.0  китайская компания mobvoi опубликовала официал...,
                                                  text_sw
 18360  глава форума reddit стив хаффман раскритиковал...
 18361  французское архитектурное бюро vincent calleba...)

In [56]:
train.shape, test.shape

((18360, 2), (1102, 1))

In [57]:
X = train['text_sw']
y = train['source']

In [58]:
type(train['text_sw'][0])

str

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [71]:
my_tags = train['source'].unique().astype(int)
my_tags

array([1, 0, 2])

In [61]:
type(X_train[1])

str

In [62]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Naive Bayes Classifier

In [65]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [66]:
%%time
nb.fit(X_train, y_train)

CPU times: user 4.68 s, sys: 21.4 ms, total: 4.7 s
Wall time: 4.7 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [67]:
%%time
y_pred = nb.predict(X_test)

CPU times: user 1.62 s, sys: 1.82 ms, total: 1.62 s
Wall time: 1.62 s


In [68]:
y_pred[0]

1.0

In [69]:
print(X_test[0], y_test[0], y_pred[0])

официальный аккаунт playstation опубликовал трейлер vr игры the walking dead onslaught сюжет основан сериале ходячие мертвецы игроку предложат сыграть одного оригинальных персонажей шоу включая рика дэрила сюжету придётся отстреливаться отбиваться холодным оружием полчищ зомби выполняя различные задания спасения александрии помимо придётся прокачивать оружие меняться ресурсами развивать поселение релиз the walking dead onslaught состоится сентября стоимость steam составляет рубля игра доступна playstation vr пк 1.0 1.0


In [73]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8440450254175744


# Linear Support Vector Machine

In [74]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [75]:
%%time
sgd.fit(X_train, y_train)

CPU times: user 5.02 s, sys: 528 ms, total: 5.55 s
Wall time: 4.76 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [76]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 1.58 s, sys: 5.58 ms, total: 1.59 s
Wall time: 1.59 s


In [77]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.9030501089324618
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.90      1039
         1.0       0.89      1.00      0.94      3511
         2.0       0.98      0.58      0.73       958

    accuracy                           0.90      5508
   macro avg       0.93      0.82      0.86      5508
weighted avg       0.91      0.90      0.90      5508



# Итоговое предсказание и результат

In [86]:
test

,text_sw
18360,глава форума reddit стив хаффман раскритиковал...
18361,французское архитектурное бюро vincent calleba...
18362,samsung провела мероприятие galaxy unpacked ра...
18363,бета версии клиента telegram android индексом ...
18364,журналистам xda удалось раздобыть минимальные ...
...,...
19457,декабря россии блокируется крупнейших торрент ...
19458,первые упоминания разработки тёмной темы googl...
19459,компания oppo презентовала новых продукта бесп...
19460,основой презентации стала windows которая тече...


In [87]:
test['source'] = sgd.predict(test.text_sw)

In [88]:
test.head()

,text_sw,source
18360,глава форума reddit стив хаффман раскритиковал...,0.0
18361,французское архитектурное бюро vincent calleba...,1.0
18362,samsung провела мероприятие galaxy unpacked ра...,1.0
18363,бета версии клиента telegram android индексом ...,1.0
18364,журналистам xda удалось раздобыть минимальные ...,1.0


In [89]:
test['source'].to_csv('result.csv', index = False)